In [7]:
%pip install "git+https://github.com/ScierKnave/TorchMPS.git"
!git clone "https://github.com/ScierKnave/honor_project.git"
import requests
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
%pip install torchmetrics
from torchmetrics.classification import MulticlassAccuracy
from torchmps import MPS
import math

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ScierKnave/TorchMPS.git to /tmp/pip-req-build-853n372p
  Running command git clone --filter=blob:none --quiet https://github.com/ScierKnave/TorchMPS.git /tmp/pip-req-build-853n372p
  Resolved https://github.com/ScierKnave/TorchMPS.git to commit f716a08e15d0af50dbfdfc435ab9604e82562ea3
  Preparing metadata (setup.py) ... done
fatal: destination path 'honor_project' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Hyperparameters

In [8]:
# Hardware hyperparameters
chosen_device = torch.device('cuda' 
if torch.cuda.is_available() else 'cpu')

# Data hyperparameters
nb_train_HP = 2000
nb_test_HP = 500
batch_sz_HP = 150
batch_sz_HP = min(batch_sz_HP, nb_train_HP)
nb_classes_HP = 10

# Student hyperparameters
# MPS parameters
bond_dim_HP = 10
# USING CUSTOM FEATURE MAP WITH FORK?: YES

# Training parameters
nepochs_student_HP = 30 
student_lr_HP = 1e-4
student_reg_HP = 0
student_loss_HP = nn.CrossEntropyLoss()


# Premilinaries: Importing the data and utils subroutines

In [9]:

# Import the *common* mnist train set and create a batch iterator for it.
train_set = torch.load('/content/honor_project/src/datasets/train_mnist.pt')
train_iterator = torch.utils.data.DataLoader(
    dataset = train_set, 
    sampler = torch.utils.data.SubsetRandomSampler(range(nb_train_HP)),
    batch_size=batch_sz_HP
    )

# Import the mnist *common* test set and create a batch iterator for it.
test_set = torch.load('/content/honor_project/src/datasets/test_mnist.pt')
test_iterator = torch.utils.data.DataLoader(
    dataset = test_set, 
    sampler = torch.utils.data.SubsetRandomSampler(range(nb_test_HP)),
    batch_size = batch_sz_HP
    )

In [10]:
# Returns the validation set classification accuracy
# of the given input model (this is a higher order function)
def get_acc(model, iterator, dataset_size):
    # Get the validation set classification accuracy
    total_good_classifications = 0
    acc_metric = MulticlassAccuracy(num_classes=nb_classes_HP).to(chosen_device)
    for (x_mb, y_mb) in iterator:
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        y_mb = y_mb.to(chosen_device)
        # Add the number of datapoints we classified right to the total
        batch_size = x_mb.size()[0]
        y_hat = model(x_mb)
        batch_good_classifications = batch_size * acc_metric(y_hat, y_mb)
        total_good_classifications += batch_good_classifications
    return total_good_classifications / dataset_size # divide by total size

# Training the student model

In [11]:
def train_a_student():
  '''
  Trains a student model.
  '''

  # Initialize the MPS module
  student = MPS(
      input_dim = 28 ** 2,
      output_dim = 10,
      bond_dim = bond_dim_HP
  ).to(chosen_device) 
  #student.register_feature_map(feature_map_HP)

  # Instantiate the optimizer and softmax
  student_optimizer = torch.optim.Adam(
      student.parameters(), lr = student_lr_HP, weight_decay = student_reg_HP
  )

  # Used on the inputs before the loss function
  LogSoftmax = nn.LogSoftmax(dim=1)

  # Create an array to store the val loss
  # of the student at each epoch
  stud_test_loss = np.array([])
  stud_train_loss = np.array([])

  softmax = nn.Softmax(dim=1)

  stud_train_loss = np.array([])
  stud_test_loss = np.array([])

  for epoch in range(nepochs_student_HP):
      '''
      Training loop through the epochs.
      '''
      for (x_mb, y_mb) in train_iterator:
          # Flatten the MNIST images, which come in matrix form
          x_mb = x_mb.reshape(-1, 784).to(chosen_device)
          y_mb = y_mb.to(chosen_device)

          student_logits = student(x_mb) 

          # Backpropagation
          loss = student_loss_HP( student_logits, y_mb)

          student_optimizer.zero_grad()
          loss.backward()
          student_optimizer.step()

      # Get accuracy over all test and training data for current epoch
      train_current_accuracy = round( get_acc(student, train_iterator, nb_train_HP).item(), 4)
      test_current_accuracy = round( get_acc(student, test_iterator, nb_test_HP).item(), 4)
      stud_train_loss = np.append(stud_train_loss, train_current_accuracy)
      stud_test_loss = np.append(stud_test_loss, test_current_accuracy)
  return(stud_train_loss, stud_test_loss)


# Repeat the training process in order to get the variance
global_stud_test_loss = np.array([])
global_stud_train_loss = np.array([])
for i in range(20):
  print("\n Training and testing of model ", i+1, " in progress...")
  (stud_train_loss, stud_test_loss) = train_a_student()
  print("Train loss: ", stud_train_loss)
  print("Test loss: ", stud_test_loss)
  if (i == 0):
    global_stud_train_loss = stud_train_loss
    global_stud_test_loss = stud_test_loss
  else:
    global_stud_train_loss = np.vstack((global_stud_train_loss, stud_train_loss))
    global_stud_test_loss = np.vstack((global_stud_test_loss, stud_test_loss))




 Training and testing of model  1  in progress...
Train loss:  [0.1    0.1005 0.1    0.1    0.1916 0.4261 0.5581 0.6735 0.7872 0.8257
 0.849  0.8742 0.8766 0.8987 0.9122 0.9246 0.9166 0.8966 0.938  0.9314
 0.95   0.9385 0.9525 0.9557 0.9568 0.955  0.9658 0.9787 0.9718 0.9811]
Test loss:  [0.1    0.1    0.1    0.1    0.1879 0.4382 0.562  0.6774 0.8058 0.8279
 0.8133 0.8569 0.8764 0.8872 0.8717 0.8906 0.8932 0.8592 0.9031 0.9021
 0.9197 0.8811 0.8933 0.9106 0.8887 0.8833 0.9008 0.8999 0.909  0.9179]

 Training and testing of model  2  in progress...
Train loss:  [0.1    0.1    0.2145 0.3525 0.3743 0.4725 0.5279 0.6138 0.7287 0.8229
 0.8509 0.8775 0.8885 0.9054 0.9073 0.92   0.9293 0.9054 0.9341 0.9328
 0.9388 0.9601 0.9497 0.9636 0.9633 0.9656 0.9743 0.9674 0.9674 0.9657]
Test loss:  [0.1    0.1    0.2128 0.3604 0.3905 0.4506 0.4872 0.5887 0.7088 0.8135
 0.8239 0.8638 0.836  0.8773 0.8669 0.9014 0.9036 0.8769 0.8996 0.8594
 0.8877 0.8947 0.8694 0.9031 0.8685 0.9082 0.9091 0.9052 0.898  

In [12]:
# Print the final results and save them for the report.
print("Final results")
print("Train loss: ", global_stud_train_loss)
print("Test loss: ", global_stud_test_loss)

np.save('20x_mps_trainloss_10bd', stud_train_loss)
np.save('20x_mps_testloss_10bd', stud_test_loss)


Final results
Train loss:  [[0.1    0.1005 0.1    0.1    0.1916 0.4261 0.5581 0.6735 0.7872 0.8257
  0.849  0.8742 0.8766 0.8987 0.9122 0.9246 0.9166 0.8966 0.938  0.9314
  0.95   0.9385 0.9525 0.9557 0.9568 0.955  0.9658 0.9787 0.9718 0.9811]
 [0.1    0.1    0.2145 0.3525 0.3743 0.4725 0.5279 0.6138 0.7287 0.8229
  0.8509 0.8775 0.8885 0.9054 0.9073 0.92   0.9293 0.9054 0.9341 0.9328
  0.9388 0.9601 0.9497 0.9636 0.9633 0.9656 0.9743 0.9674 0.9674 0.9657]
 [0.1    0.1018 0.3002 0.4481 0.6171 0.6874 0.7857 0.8069 0.8594 0.8548
  0.8698 0.8924 0.9075 0.9115 0.9043 0.9172 0.9269 0.9217 0.9399 0.937
  0.9489 0.9554 0.9188 0.9171 0.9569 0.9428 0.9679 0.9686 0.97   0.9688]
 [0.1687 0.3579 0.4105 0.4769 0.633  0.734  0.7955 0.8073 0.8704 0.9056
  0.92   0.913  0.9059 0.9363 0.9375 0.9415 0.9458 0.9259 0.9541 0.9423
  0.967  0.9661 0.9674 0.9755 0.9665 0.977  0.9849 0.9634 0.9808 0.9858]
 [0.1724 0.1767 0.4439 0.496  0.6473 0.7396 0.7802 0.8243 0.8693 0.8713
  0.8849 0.8828 0.9104 0.9143 0.92